In [ ]:
###### Yahoo Finance Market News -- Keep

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import os

def get_market_news(tickers=None, days_back=7):
    """
    Retrieve market news from Yahoo Finance API.
    
    Parameters:
    tickers (list): List of ticker symbols to get news for. If None, gets general market news
    days_back (int): Number of days of historical news to retrieve
    
    Returns:
    pandas.DataFrame: DataFrame containing news items
    """
    # Initialize empty list to store news
    all_news = []
    
    if tickers is None:
        # Use ^GSPC (S&P 500) for general market news
        tickers = ['^GSPC']
    
    for ticker in tickers:
        # Create ticker object
        stock = yf.Ticker(ticker)
        
        try:
            # Get news for the ticker
            news = stock.news
            
            # Process each news item
            for item in news:
                news_item = {
                    'ticker': ticker,
                    'title': item.get('title'),
                    'publisher': item.get('publisher'),
                    'link': item.get('link'),
                    'publish_date': datetime.fromtimestamp(item.get('providerPublishTime', 0)),
                    'type': item.get('type'),
                    'related_tickers': ', '.join(item.get('relatedTickers', []))
                }
                all_news.append(news_item)
                
        except Exception as e:
            print(f"Error retrieving news for {ticker}: {str(e)}")
    
    # Convert to DataFrame and sort by date
    news_df = pd.DataFrame(all_news)
    if not news_df.empty:
        news_df = news_df.sort_values('publish_date', ascending=False)
        
        # Filter for recent news only
        cutoff_date = datetime.now() - timedelta(days=days_back)
        news_df = news_df[news_df['publish_date'] >= cutoff_date]
    
    return news_df

def save_news_to_csv(news_df, output_dir='market_news', filename_prefix='market_news'):
    """
    Save news DataFrame to CSV with timestamp in filename.
    
    Parameters:
    news_df (pandas.DataFrame): DataFrame containing news items
    output_dir (str): Directory to save CSV files
    filename_prefix (str): Prefix for the CSV filename
    
    Returns:
    str: Path to the saved CSV file
    """
    try:
        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"Created directory: {output_dir}")
        
        # Generate filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{filename_prefix}_{timestamp}.csv"
        filepath = os.path.join(output_dir, filename)
        
        # Save to CSV
        news_df.to_csv(filepath, index=False, encoding='utf-8')
        print(f"Successfully saved news to: {filepath}")
        
        return filepath
        
    except Exception as e:
        print(f"Error saving CSV file: {str(e)}")
        return None

def main():
    # Create output directory name based on date
    today = datetime.now().strftime('%Y%m%d')
    output_dir = f"market_news_{today}"
    
    # Get general market news
    market_news = get_market_news()
    if not market_news.empty:
        print("\nGeneral Market News:")
        print(market_news[['publish_date', 'title', 'publisher']].head())
        # Save general market news
        save_news_to_csv(market_news, output_dir, 'general_market_news')
    
    # Get news for specific stocks
    tech_stocks = ['AAPL', 'GOOGL', 'MSFT']
    tech_news = get_market_news(tickers=tech_stocks, days_back=3)
    if not tech_news.empty:
        print("\nTech Stocks News (Last 3 days):")
        print(tech_news[['ticker', 'publish_date', 'title', 'publisher']].head())
        # Save tech stocks news
        save_news_to_csv(tech_news, output_dir, 'tech_stocks_news')

if __name__ == "__main__":
    main()

In [38]:
###################### AphaVantage Market News - Currently set to Russell 200 so need to change.  Last run gave me day old news
import requests
import json
import pandas as pd
from datetime import datetime
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_market_news(api_key, keywords="none", limit=10):
    """
    Retrieve market news from Alpha Vantage API.
    
    Parameters:
    api_key (str): Alpha Vantage API key
    keywords (str): Search keywords
    limit (int): Maximum number of news items to retrieve
    
    Returns:
    list: List of news items
    """
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'NEWS_SENTIMENT',
        'apikey': api_key,
        'keywords': keywords,
        'limit': limit
    }
    
    try:
        logger.info(f"Fetching news for keywords: {keywords}")
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        news_data = response.json()
        
        if 'Note' in news_data:
            logger.warning(f"API limit message: {news_data['Note']}")
            return None
            
        if 'feed' in news_data:
            logger.info(f"Retrieved {len(news_data['feed'])} news items")
            return news_data['feed']
        else:
            logger.warning("No news found in response")
            return None
            
    except requests.exceptions.RequestException as e:
        logger.error(f"Request failed: {str(e)}")
        return None
    except json.JSONDecodeError as e:
        logger.error(f"Failed to parse JSON response: {str(e)}")
        return None
    except Exception as e:
        logger.error(f"An unexpected error occurred: {str(e)}")
        return None

def save_news_to_csv(news_items, output_dir='russell2000_news'):
    """
    Save news items to a CSV file.
    
    Parameters:
    news_items (list): List of news items
    output_dir (str): Directory to save the CSV file
    
    Returns:
    str: Path to the saved CSV file
    """
    try:
        if not news_items:
            logger.warning("No news items to save")
            return None
            
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Process news items with proper string handling
        processed_items = []
        for item in news_items:
            processed_item = {
                'title': str(item.get('title', '')),
                'publish_date': str(item.get('time_published', '')),
                'summary': str(item.get('summary', '')),
                'url': str(item.get('url', '')),
                'source': str(item.get('source', '')),
                'overall_sentiment_score': str(item.get('overall_sentiment_score', '')),
                'overall_sentiment_label': str(item.get('overall_sentiment_label', '')),
                'topics': ', '.join(str(topic) for topic in item.get('topics', [])),
                'ticker_sentiment': json.dumps(item.get('ticker_sentiment', [])),
                'authors': ', '.join(str(author) for author in item.get('authors', []))
            }
            processed_items.append(processed_item)
        
        # Convert to DataFrame
        news_df = pd.DataFrame(processed_items)
        
        # Generate filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"russell2000_news_{timestamp}.csv"
        filepath = os.path.join(output_dir, filename)
        
        # Save to CSV
        news_df.to_csv(filepath, index=False, encoding='utf-8')
        logger.info(f"Successfully saved news to: {filepath}")
        
        # Print preview of saved data
        logger.info("\nPreview of saved data:")
        logger.info(news_df[['title', 'publish_date', 'source']].head())
        
        return filepath
        
    except Exception as e:
        logger.error(f"Error saving to CSV: {str(e)}")
        return None

def main():
    # Your API key
    api_key = 'FLGDYAANWX6EFL9P'
    
    # Get current directory for saving files
    current_dir = os.getcwd()
    output_dir = os.path.join(current_dir, 'russell2000_news')
    
    # Fetch news
    news_items = get_market_news(api_key, limit=50)  # Increased limit for more data
    
    if news_items:
        # Save to CSV
        saved_file = save_news_to_csv(news_items, output_dir)
        if saved_file:
            logger.info(f"News data has been saved to: {saved_file}")
    else:
        logger.warning("No news data was retrieved")

if __name__ == "__main__":
    try:
        logger.info("Starting Russell 2000 news retrieval process")
        main()
        logger.info("Process completed")
    except Exception as e:
        logger.error(f"Main process error: {str(e)}")

2024-11-21 12:51:49,460 - INFO - Starting Russell 2000 news retrieval process
2024-11-21 12:51:49,464 - INFO - Fetching news for keywords: none
2024-11-21 12:51:49,789 - INFO - Retrieved 50 news items
2024-11-21 12:51:49,818 - INFO - Successfully saved news to: c:\Users\BryceDaniel\OneDrive - Lincoln Telephone Company\MSBA\GitHub\TrendSense\russell2000_news\russell2000_news_20241121_125149.csv
2024-11-21 12:51:49,819 - INFO - 
Preview of saved data:
2024-11-21 12:51:49,829 - INFO -                                                title     publish_date  \
0  TX RX Systems - 50 Years of Leadership in LMR ...  20241121T192100   
1  These Analysts Boost Their Forecasts On Wix.co...  20241121T191559   
2  How Is The Market Feeling About Equifax? - Equ...  20241121T191518   
3  How Is The Market Feeling About Yum Brands? - ...  20241121T191513   
4  Williams-Sonoma  ( WSM )  technical analysis -...  20241121T191355   

     source  
0  Benzinga  
1  Benzinga  
2  Benzinga  
3  Benzinga  
4  B

In [ ]:
############### Alpha Vantage 10K financials.................
import requests
import pandas as pd

def get_alpha_vantage_10k_data(api_key, symbol, output_file="10k_financial_data.csv"):
    """
    Fetch key financial data (Income Statement, Balance Sheet, Cash Flow) for a company using Alpha Vantage API.
    
    Parameters:
        api_key (str): Your Alpha Vantage API key.
        symbol (str): The stock ticker symbol (e.g., "AAPL" for Apple).
        output_file (str): Path to save the financial data as a CSV.
    
    Returns:
        None: Saves financial data to a CSV file.
    """
    BASE_URL = "https://www.alphavantage.co/query"
    endpoints = {
        "income_statement": "INCOME_STATEMENT",
        "balance_sheet": "BALANCE_SHEET",
        "cash_flow": "CASH_FLOW"
    }
    
    financial_data = {}
    
    for report_type, function in endpoints.items():
        print(f"Fetching {report_type.replace('_', ' ').title()}...")
        
        params = {
            "function": function,
            "symbol": symbol,
            "apikey": api_key
        }
        
        try:
            response = requests.get(BASE_URL, params=params)
            response.raise_for_status()
            data = response.json()
            
            if f"annualReports" in data:
                # Extract the most recent annual report
                financial_data[report_type] = data["annualReports"]
            else:
                print(f"Warning: No data found for {report_type}.")
        
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {report_type}: {e}")
    
    # Combine all financial data into a single DataFrame
    combined_data = []
    for report_type, reports in financial_data.items():
        for report in reports:
            report["type"] = report_type
            combined_data.append(report)
    
    # Convert combined data to a DataFrame
    if combined_data:
        df = pd.DataFrame(combined_data)
        df.to_csv(output_file, index=False)
        print(f"Financial data saved to {output_file}")
    else:
        print("No financial data to save.")

# Example Usage
if __name__ == "__main__":
    API_KEY = "FLGDYAANWX6EFL9P"  # Replace with your Alpha Vantage API key
    SYMBOL = "AAPL"  # Example: Apple Inc.
    get_alpha_vantage_10k_data(API_KEY, SYMBOL, output_file="aapl_10k_financial_data.csv")

Fetching Income Statement...
Fetching Balance Sheet...
Fetching Cash Flow...
Financial data saved to aapl_10k_financial_data.csv


Retrieving executive narratives or specific sections (like "Management’s Discussion and Analysis" or "Risk Factors") from 10-K filings requires parsing the full text of the filings. While Alpha Vantage does not support text-based 10-K narrative retrieval, the SEC EDGAR system does. Here's how you can retrieve and parse these sections:

Approach to Retrieve Executive Narratives from 10-K
1. Using SEC EDGAR API
The SEC EDGAR system allows direct access to company filings, including 10-Ks. You can download the full text of a filing in HTML format and parse it to extract specific sections.

In [ ]:
################  Analyzing 10K narrative.  Need to fix
import requests
from bs4 import BeautifulSoup

def fetch_10k_filing(company_cik, filing_type="10-K", year="2023"):
    """
    Fetch the URL for a company's 10-K filing from SEC EDGAR.

    Parameters:
        company_cik (str): CIK (Central Index Key) for the company.
        filing_type (str): Filing type to search for (default: "10-K").
        year (str): Filing year (default: "2023").

    Returns:
        str: Filing document URL or None if not found.
    """
    BASE_URL = f"https://data.sec.gov/submissions/CIK{company_cik}.json"
    headers = {"User-Agent": "YourName email@example.com"}
    
    response = requests.get(BASE_URL, headers=headers)
    if response.status_code == 200:
        data = response.json()
        filings = data.get("filings", {}).get("recent", {})
        
        # Find the most recent 10-K filing for the specified year
        for i in range(len(filings["formType"])):
            if filings["formType"][i] == filing_type and year in filings["filingDate"][i]:
                accession_number = filings["accessionNumber"][i].replace("-", "")
                doc_url = f"https://www.sec.gov/Archives/edgar/data/{company_cik}/{accession_number}/{filings['primaryDocument'][i]}"
                return doc_url
    else:
        print(f"Failed to fetch filings for CIK {company_cik}. Status code: {response.status_code}")
    return None


def extract_section_from_10k(url, section_title):
    """
    Extract a specific section from a 10-K filing document.

    Parameters:
        url (str): URL to the 10-K filing document.
        section_title (str): Title of the section to extract (e.g., "Management’s Discussion and Analysis").

    Returns:
        str: Extracted section text or None if not found.
    """
    headers = {"User-Agent": "YourName email@example.com"}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract text content and find the section
        text = soup.get_text(separator="\n")
        lines = text.splitlines()
        
        # Locate the section by title
        section_text = []
        capture = False
        for line in lines:
            if section_title.lower() in line.lower():
                capture = True  # Start capturing lines after finding the title
            elif capture and line.strip() == "":
                break  # Stop capturing at the next blank line
            if capture:
                section_text.append(line)
        
        return "\n".join(section_text) if section_text else None
    else:
        print(f"Failed to fetch 10-K document. Status code: {response.status_code}")
        return None


# Example Usage
if __name__ == "__main__":
    CIK = "0000320193"  # Apple Inc. CIK
    YEAR = "2023"
    SECTION_TITLE = "Management’s Discussion and Analysis"
    
    # Step 1: Get the filing document URL
    filing_url = fetch_10k_filing(CIK, year=YEAR)
    if filing_url:
        print(f"10-K Filing URL: {filing_url}")
        
        # Step 2: Extract the specific section
        narrative = extract_section_from_10k(filing_url, SECTION_TITLE)
        if narrative:
            print(f"Extracted Section ({SECTION_TITLE}):\n")
            print(narrative)
        else:
            print(f"Section '{SECTION_TITLE}' not found.")
    else:
        print("10-K filing not found.")



KeyError: 'formType'